In [1]:
!pip3 install -q tensorflow_decision_forests

     |████████████████████████████████| 13.4 MB 316 kB/s 
     |████████████████████████████████| 462 kB 30.9 MB/s 


In [2]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle competitions download -c nlp-getting-started

  0% 0.00/593k [00:00<?, ?B/s]
100% 593k/593k [00:00<00:00, 85.9MB/s]


In [4]:
import zipfile
with zipfile.ZipFile("/content/nlp-getting-started.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/")

In [5]:
import pandas as pd
import numpy as np
import re

import tensorflow as tf
import tensorflow_decision_forests as tfdf
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from tensorflow.keras.regularizers import l2, l1_l2
import tensorflow_hub as hub

In [6]:
# Turn .csv files into pandas DataFrame's
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
train_df.shape

(7613, 5)

In [8]:
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.drop(['id','keyword','location'],axis=1,inplace=True)
train_df_shuffled.head()

,text,target
2644,So you have a new weapon that can cause un-ima...,1
2227,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,Aftershock back to school kick off was great. ...,0
6845,in response to trauma Children of Addicts deve...,0


In [9]:
train_df_shuffled.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7613 entries, 2644 to 7270
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    7613 non-null   object
 1   target  7613 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 178.4+ KB


In [10]:
train_df_shuffled.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [11]:
# How many samples total?
print(f"Total training samples: {len(train_df)}")
print(f"Total test samples: {len(test_df)}")
print(f"Total samples: {len(train_df) + len(test_df)}")

Total training samples: 7613
Total test samples: 3263
Total samples: 10876


In [12]:
import random
for i in range(5):
    random_index = random.randint(0, len(train_df))
    target = train_df_shuffled.iloc[random_index][1]
    text = train_df_shuffled.iloc[random_index][0]

    if target==1: print(f'Target: {target} (real disaster)')
    else: print(f'Target: {target} (not real disaster)')
    print(f'Text: {text}\n')
    print('-'*10)

Target: 0 (not real disaster)
Text: @Eric_Bulak @jaclynsonne @_OliviaAnn_ I was looking for you guys on the live stream. I'm guessing the evacuation cost you the front?

----------
Target: 1 (real disaster)
Text: 1 hour parade like 50 people collapsed. #OneHeartOneMindOneCSS

----------
Target: 1 (real disaster)
Text: Families to sue over Legionnaires: More than 40 families affected by the fatal outbreak of Legionnaires' disea... http://t.co/VBsj8tniv1

----------
Target: 0 (not real disaster)
Text: Official kinesiology tape of IRONMANå¨ long-lasting durability effectiveness on common injuries http://t.co/ejymkZPEEx http://t.co/0IYuntXDUv

----------
Target: 0 (not real disaster)
Text: Choking Hazard Prompts Recall Of Kraft Cheese Singles http://t.co/XGKyVF9t4f

----------


In [13]:
from sklearn.model_selection import train_test_split

# Use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # dedicate 10% of samples to validation set
                                                                            random_state=42) # random state for reproducibility

In [14]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_sentences, train_labels))
valid_dataset = tf.data.Dataset.from_tensor_slices((val_sentences, val_labels))

train_dataset

<TensorSliceDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [15]:
train_dataset = train_dataset.batch(32).prefetch(tf.data.AUTOTUNE)
valid_dataset = valid_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

In [17]:
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[],
                                        dtype=tf.string,
                                        trainable=False,
                                        name="USE")

In [18]:
inputs = tf.keras.layers.Input(shape=(), dtype=tf.string)
x = sentence_encoder_layer(inputs)
preprocessor = tf.keras.Model(inputs=inputs, outputs=x)

model = tfdf.keras.GradientBoostedTreesModel(preprocessing=preprocessor,num_trees=1000)
model.compile(metrics=["accuracy"])

model.fit(x=train_dataset)

print("Evaluation:", model.evaluate(valid_dataset))

Use /tmp/tmphbompeim as temporary training directory
Starting reading the dataset
215/215 [==============================] - ETA: 0s
Dataset read in 0:00:52.395502
Training model
Model trained in 0:02:51.251765
Compiling model
215/215 [==============================] - 224s 928ms/step
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
24/24 [==============================] - 3s 24ms/step - loss: 0.0000e+00 - accuracy: 0.8163
Evaluation: [0.0, 0.8162729740142822]


In [19]:
print("Evaluation :", model.evaluate(valid_dataset))

24/24 [==============================] - 2s 24ms/step - loss: 0.0000e+00 - accuracy: 0.8163
Evaluation : [0.0, 0.8162729740142822]


In [20]:
def calculate_results(y_true, y_pred):
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

In [21]:
preds = model.predict(val_sentences)
preds = tf.squeeze(tf.round(preds))
preds.shape, preds[:10]
result = calculate_results(y_true=val_labels,
                                    y_pred=preds)
result

{'accuracy': 81.49606299212599,
 'f1': 0.812164431568833,
 'precision': 0.8218492392339359,
 'recall': 0.8149606299212598}